In [15]:
import torch.utils.data
import sklearn.metrics
import torch
import pandas
from torch.utils.data import Dataset
import tqdm
import spacy
import random
import numpy as np
import math
import os
import torchvision 
from torchvision import models, datasets, transforms
SEED = 2222
random.seed(SEED)
torch.manual_seed(SEED)
nlp = spacy.load('en')
pandas.read_csv('train.csv',header=0).iloc[0]

id                                                          1
keyword                                                   NaN
location                                                  NaN
text        Our Deeds are the Reason of this #earthquake M...
target                                                      1
Name: 0, dtype: object

In [16]:
pandas.read_csv('test.csv',header=0).iloc[0]

id                                           0
keyword                                    NaN
location                                   NaN
text        Just happened a terrible car crash
Name: 0, dtype: object

In [17]:
class TrainDataset(Dataset):
    def __init__(self):
        self.data = pandas.read_csv('train.csv',header=0)
            
            
        self.ordinals = {}
        for sample in tqdm.tqdm(self.data.text):
            for token in nlp (sample.lower(),disable = ['parser','tagger','ner']):
                if token.text not in self.ordinals:
                    self.ordinals[token.text] = len(self.ordinals)
    def __len__ (self):
        return len(self.data)
    def __getitem__(self,idx):
        if type(idx) is torch.Tensor:
            idx = idx.item()
        sample = self.data.iloc[idx]
        bag_of_words = torch.zeros(len(self.ordinals))
        for token in nlp (sample.text.lower(),disable = ['parser','tagger','ner']):
            bag_of_words[self.ordinals[token.text]] += 1
        return bag_of_words, torch.tensor(sample.target)


In [18]:
class TestDataset(Dataset):
    def __init__(self):
        self.data = pandas.read_csv('test.csv',header=0)
            
            
        self.ordinals = {}
        for sample in tqdm.tqdm(self.data.text):
            for token in nlp (sample.lower(),disable = ['parser','tagger','ner']):
                if token.text not in self.ordinals:
                    self.ordinals[token.text] = len(self.ordinals)
        #print('efsfgsdfsdf',len(self.ordinals))
    def __len__ (self):
        return len(self.data)
    def __getitem__(self,idx):
        if type(idx) is torch.Tensor:
            idx = idx.item()
        sample = self.data.iloc[idx]
        bag_of_words = torch.zeros(22610)
        for token in nlp (sample.text.lower(),disable = ['parser','tagger','ner']):
            bag_of_words[self.ordinals[token.text]] += 1
        return bag_of_words

In [19]:
class Modeltrain (torch.nn.Module):
    def __init__(self, input_dimensions,size=128):
        super().__init__()
        self.layer_one = torch.nn.Linear(input_dimensions, size)
        self.activation_one = torch.nn.ReLU()
        self.layer_two = torch.nn.Linear(size, size)
        self.activation_two = torch.nn.ReLU()
        self.shape_outputs = torch.nn.Linear(size, 2)

    def forward(self, inputs):

        buffer = self.layer_one(inputs)
        buffer = self.activation_one(buffer)
        buffer = self.layer_two(buffer)
        buffer = self.activation_two(buffer)
        buffer = self.shape_outputs(buffer)
        return buffer


In [20]:

def valid(model, validationloader):
  loss_all = []
  with torch.no_grad():
      model.eval()
      for inputs, actual in validationloader:
          results = model(inputs)
          loss = loss_function(results, actual)
          loss_all.append(loss.item())
  return  np.mean(np.array(loss_all))

def train(model, trainloader, optimizer, loss_function):
  model.train()
  epoch_loss = []
  for inputs, outputs in trainloader:
        optimizer.zero_grad()
        results = model(inputs)
        loss = loss_function(results, outputs)
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
  return np.mean(np.array(epoch_loss))


In [ ]:
#training dataset
traindataset = TrainDataset()
number_for_validating = int(len(traindataset)*0.1)
number_for_training = len(traindataset) - number_for_validating
valid_dataset, train_dataset = torch.utils.data.random_split(traindataset,[
    number_for_validating,number_for_training])
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size =16, shuffle =True)
validationloader = torch.utils.data.DataLoader(valid_dataset, batch_size =16, shuffle =True)
print(len(valid_dataset),len(train_dataset),len(trainloader),len(validationloader))
#testing dataset
testdataset = TestDataset()
test_number = int(len(testdataset))
print(test_number)
test_loader = torch.utils.data.DataLoader(testdataset,batch_size =16, shuffle =True)
#loss
model = Modeltrain(len(traindataset.ordinals))
optimizer = torch.optim.Adam(modeltrain.parameters())
loss_function = torch.nn.CrossEntropyLoss()
best_loss = float ('inf')
best_epoch = 0
for epoch in range(50):
    train_loss = train(model, trainloader, optimizer, loss_function)
    valid_loss = valid(model, validationloader)
    if valid_loss<best_loss:
        best_loss=valid_loss
        best_epoch = epoch
        torch.save(modeltrain.state_dict(),'best_model.pt')
    print('Epoch:%d, Current_loss:%.4f Best_epoch:%d Best_loss:%.4f'%(epoch,valid_loss,best_epoch, best_loss))

 11%|█         | 347/3263 [00:00<00:00, 3464.77it/s]

761 6852 429 48


100%|██████████| 3263/3263 [00:01<00:00, 2642.18it/s]


3263


In [48]:
model.load_state_dict(torch.load('best_model.pt'))
results= []
with torch.no_grad():
    model.eval()

    for inputs in test_loader:
        result = modeltrain(inputs).argmax(dim=1).numpy()
        results.append(result)
results = np.hstack(results)
print(results.shape)

(3263,)


In [0]:
np.savetxt("foo.csv", results.astype(int), fmt='%d')